In [114]:
#!pip install selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import re,csv,time
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

In [132]:
def scrape(url):
    #create a variable (wdpath) to store the chrome web driver
    wdpath = "chromedriver.exe"
    #create a variable named "driver" to store the actual browser being used (chrome) and its corresponding webdriver(wdpath)
    driver = webdriver.Chrome(wdpath)
    
    reviewer_name = []
    review_date = []
    review_text = []
    review_polarity = []
    
    driver.get(url)
    driver.maximize_window()
    
    while True:
        #script to scroll down to the end of the page
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight)')
        
        #retrieve all the reviews for the movie
        reviews = driver.find_elements_by_css_selector('[class= "row review_table_row"]')
        
        for review in reviews:

            #Reviewer name
            try:
                name = review.find_element_by_css_selector('[class= "unstyled bold articleLink"]')
                reviewer_name.append(name.text)
            except:
                name = None

            #Review date
            try:
                dates = review.find_element_by_css_selector('[class = "review-date subtle small"]')
                review_date.append(dates.text)
            except:
                date = None

            #Review text
            try:
                text = review.find_element_by_css_selector('[class="the_review"]')
                review_text.append(text.text)
            except:
                text = None

            #Review polarity
            try:
                polarity_fresh = review.find_element_by_css_selector('[class = "review_icon icon small fresh"]')
                find = polarity_fresh.get_attribute('class')
                n = re.search("review_icon icon small (fresh)",find)
                review_polarity.append(n.group(1))
            except:
                polarity_rotten = review.find_element_by_css_selector('[class = "review_icon icon small rotten"]')
                review_polarity.append('rotten') 


        nextbutton = WebDriverWait(driver,5).until(EC.presence_of_element_located((By.CLASS_NAME,'js-prev-next-paging-next')))

        if 'hide' in nextbutton.get_attribute('class'):
            break
                
        nextbutton.click()

        time.sleep(3)
            
    driver.close()
            
    file = pd.DataFrame({"reviewer's name":reviewer_name, "review date":review_date, "review text":review_text, "review_polarity":review_polarity})
    file.to_csv('output.csv', index = False)  

In [133]:
url = 'https://www.rottentomatoes.com/m/exodus_gods_and_kings/reviews'
scrape(url)

C:\Users\Bilal\AppData\Local\Temp/ipykernel_33560/572894203.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(wdpath)
C:\Users\Bilal\AppData\Local\Temp/ipykernel_33560/572894203.py:20: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  reviews = driver.find_elements_by_css_selector('[class= "row review_table_row"]')
